# 01 - Dr. No

In [1]:
import tensorflow as tf
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import time
import sys
import io

from sklearn.model_selection import train_test_split

from nn_helpers import *

%config InlineBackend.figure_format = 'retina'
darkness = True

/Users/andrew/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Log Number - 517548

In [2]:
# log_random_serial_number = '{}'.format(np.random.randint(100000,999999))
# print(log_random_serial_number)

In [3]:
log_direc = log_dir('517548')
print(log_direc)

../../../logs/517548G


In [4]:
flags = tf.app.flags

# flag 'f' prevents an error due to using flags in jupyter notebook
#https://github.com/tensorflow/tensorflow/issues/17702#issuecomment-387335646
flags.DEFINE_string('f', '', 'kernel')

# Data parameters
flags.DEFINE_float("testing_data_percentage", 0.20,"Percentage of the data to use for testing. (default: 0.10)")
flags.DEFINE_string('data_path', '../../../data/wrapped_2500_zero_noise_A.bz2', "Data source for training and testing")

# training parameters
flags.DEFINE_integer('num_epochs', 1000, "Number of training epochs")
flags.DEFINE_integer('num_batches', 125, "Number of batches per epoch")
flags.DEFINE_float('keep_prob', 0.50, "Dropout probability")
flags.DEFINE_float('learning_rate', 0.00015, "Learning rate")

# saving parameters
flags.DEFINE_string('log_directory', log_direc, "Log save directory")
flags.DEFINE_integer('image_save_interval', 5, "Epoch to save images on.")

FLAGS = tf.flags.FLAGS



In [5]:
data = load_data(FLAGS.data_path)
scaled_input, scaled_output, input_scaler, output_scaler = scale_data(data)
X_train, X_test, y_train, y_test = train_test_split(scaled_input, scaled_output,
                                                    test_size = FLAGS.testing_data_percentage,
                                                    random_state = np.random.seed(int(time.time())))

In [6]:
def gen_plot(predicted_values, actual_values, ylims = [-100,100]):
    """Create a pyplot plot and save to buffer."""
    prediction_unscaled = output_scaler.inverse_transform(predicted_values)
    actual_unscaled = output_scaler.inverse_transform(actual_values)

    sorting_idx = np.argsort(actual_unscaled.T[0])
        
    fig, ax = plt.subplots(figsize = (5, 3), dpi = 144)

    ax.plot(prediction_unscaled.T[0][sorting_idx], linestyle = 'none', marker = '.', markersize = 1, color = 'darkblue')
    ax.plot(actual_unscaled.T[0][sorting_idx],     linestyle = 'none', marker = '.', markersize = 1, color = '#E50000', alpha = 0.50, )        

    ax.set_ylim(ylims[0],ylims[1])

    buf = io.BytesIO()
    fig.savefig(buf, format='png', dpi = 144)
    plt.close(fig)
    buf.seek(0)

    return buf.getvalue()

In [7]:
number_of_inputs  = scaled_input.shape[1]
number_of_outputs = scaled_output.shape[1]

# these should probably become flags as well
layer_1_nodes = 115
layer_2_nodes = 280
layer_3_nodes = 450
layer_4_nodes = 290
layer_5_nodes = 205
layer_6_nodes = 105
layer_7_nodes = 50

In [8]:
tf.reset_default_graph()

In [9]:
# Input Layer
with tf.variable_scope('input'):
    X  = tf.placeholder(tf.float32, shape = (None, number_of_inputs))
    
with tf.variable_scope('input_y'):
    y  = tf.placeholder(tf.float32, shape = (None, 1))
    
# for dropout keep_prob = 0.5, for testing keep_prob = 1
with tf.variable_scope('keep'):
    keep_prob = tf.placeholder(tf.float32)

# Layer 1
with tf.variable_scope('layer_1'):

    biases = tf.get_variable(name = "biases1",
                             shape = [layer_1_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights1",
                              shape  = [number_of_inputs, layer_1_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_1_output = tf.nn.leaky_relu(tf.matmul(X, weights) + biases)
    
with tf.variable_scope('dropout_12'):
    drop_out_12 = tf.nn.dropout(layer_1_output, keep_prob)     
    
    
# Layer 2
with tf.variable_scope('layer_2'):

    biases = tf.get_variable(name = "biases2",
                             shape = [layer_2_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights2",
                              shape  = [layer_1_nodes, layer_2_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_2_output = tf.nn.relu(tf.matmul(drop_out_12, weights) + biases)
    
with tf.variable_scope('dropout_23'):
    drop_out_23 = tf.nn.dropout(layer_2_output, keep_prob)     
    


# Layer 3
with tf.variable_scope('layer_3'):

    biases = tf.get_variable(name = "biases3",
                             shape = [layer_3_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights3",
                              shape  = [layer_2_nodes, layer_3_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_3_output = tf.nn.relu(tf.matmul(drop_out_23, weights) + biases)

with tf.variable_scope('dropout_34'):
    drop_out_34 = tf.nn.dropout(layer_3_output, keep_prob)  
    
# Layer 4
with tf.variable_scope('layer_4'):

    biases = tf.get_variable(name = "biases4",
                             shape = [layer_4_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights4",
                              shape  = [layer_3_nodes, layer_4_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_4_output = tf.nn.relu(tf.matmul(drop_out_34, weights) + biases)

with tf.variable_scope('dropout_45'):
    drop_out_45 = tf.nn.dropout(layer_4_output, keep_prob)  

# Layer 5
with tf.variable_scope('layer_5'):

    biases = tf.get_variable(name = "biases5",
                             shape = [layer_5_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights5",
                              shape  = [layer_4_nodes, layer_5_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_5_output = tf.nn.relu(tf.matmul(drop_out_45, weights) + biases)

with tf.variable_scope('dropout_56'):
    drop_out_56 = tf.nn.dropout(layer_5_output, keep_prob)  
    
# Layer 6
with tf.variable_scope('layer_6'):

    biases = tf.get_variable(name = "biases6",
                             shape = [layer_6_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights6",
                              shape  = [layer_5_nodes, layer_6_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_6_output = tf.nn.relu(tf.matmul(drop_out_56, weights) + biases)
    
with tf.variable_scope('dropout_67'):
    drop_out_67 = tf.nn.dropout(layer_6_output, keep_prob) 

# Layer 7
with tf.variable_scope('layer_7'):

    biases = tf.get_variable(name = "biases7",
                             shape = [layer_7_nodes],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights7",
                              shape  = [layer_6_nodes, layer_7_nodes],
                         initializer = tf.contrib.layers.xavier_initializer())


    layer_7_output = tf.nn.relu(tf.matmul(drop_out_67, weights) + biases)


# Output layer

with tf.variable_scope('output'):

    biases = tf.get_variable(name = "biases_out",
                             shape = [number_of_outputs],
                             initializer = tf.zeros_initializer())

    weights = tf.get_variable(name = "weights_out",
                              shape  = [layer_7_nodes, number_of_outputs],
                         initializer = tf.contrib.layers.xavier_initializer())


    prediction = tf.nn.relu(tf.matmul(layer_7_output, weights) + biases)

with tf.variable_scope('cost'):

    cost = tf.sqrt(tf.reduce_mean(tf.squared_difference(prediction, y)))

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(FLAGS.learning_rate, epsilon=1e-08).minimize(cost)

with tf.variable_scope('image'):
    image_buf  = tf.placeholder(tf.string, shape=[])
    epoch_image = tf.expand_dims(tf.image.decode_png(image_buf, channels=4), 0)
    
with tf.variable_scope('logging'):
    
    tf.summary.image('prediction_vs_actual', epoch_image)
    tf.summary.histogram('predictions', prediction)
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

In [10]:
saver = tf.train.Saver()

In [11]:
with tf.Session() as session:

    session.run(tf.global_variables_initializer())
    training_writer = tf.summary.FileWriter(log_direc + '/training', session.graph)
    testing_writer = tf.summary.FileWriter(log_direc + '/testing', session.graph)
    model_save_location = log_direc + '/trained_model.ckpt'

    
    for epoch in range(FLAGS.num_epochs):
        
        split_permutation = np.random.permutation(X_train.shape[0])
        X_train_batches = np.vsplit(X_train[split_permutation], FLAGS.num_batches)
        y_train_batches = np.vsplit(y_train[split_permutation], FLAGS.num_batches)
        
        for i in range(FLAGS.num_batches):
            X_train_batch = X_train_batches[i]
            y_train_batch = y_train_batches[i]
        
            session.run(optimizer,
                        feed_dict = {X: X_train_batch, y: y_train_batch, keep_prob : 0.50})

        prediction_scaled_test = session.run(prediction, feed_dict = {X: X_train, keep_prob : 1.00})
        image_buffer = gen_plot(prediction_scaled_test, y_train, ylims = [-210,210])

            
        training_feed = {X: X_train, y: y_train, keep_prob : 1.00,image_buf: image_buffer}
        training_cost, training_summary = session.run([cost, summary], feed_dict = training_feed)

        prediction_scaled_test = session.run(prediction, feed_dict = {X: X_test, keep_prob : 1.00})
        image_buffer = gen_plot(prediction_scaled_test, y_test, ylims = [-210,210])
        
        testing_feed =  {X: X_test, y: y_test, keep_prob : 1.00,image_buf: image_buffer}
        testing_cost, testing_summary = session.run([cost, summary],feed_dict = testing_feed)
        
        sys.stdout.write('\r epoch: {:4.0f} testing_cost: {:2.10f}'.format(epoch, testing_cost))
        
        training_writer.add_summary(training_summary, epoch)
        testing_writer.add_summary(testing_summary, epoch)
        testing_writer.flush()
        training_writer.flush()

        if (epoch + 1) % 50 == 0:
            
            saver.save(session, model_save_location, epoch)

            
    saver.save(session, model_save_location, epoch)

 epoch:  111 testing_cost: 0.2825675309

KeyboardInterrupt: 